In [1]:
import numpy as np

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision.transforms import transforms

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.manual_seed(42)

In [4]:
train_dataset = datasets.CIFAR10(root='cifar10_data/', 
                                 train=True, 
                                 transform=transforms.ToTensor(), 
                                 download=True
                                )

train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=128, 
                          shuffle=True
                         )

Files already downloaded and verified


In [5]:
test_dataset = datasets.CIFAR10(root='cifar10_data/', 
                                 train=False, 
                                 transform=transforms.ToTensor(), 
                                 download=True
                                )
test_loader = DataLoader(dataset=test_dataset, 
                          batch_size=8, 
                          shuffle=True
                         )

Files already downloaded and verified


In [6]:
classes = ['plane', 'car', 'bird', 'cat','deer', 'dog', 
           'frog', 'horse', 'ship', 'truck']

In [7]:
device = 'cuda'

In [8]:
class MyLayer(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, padding=1, padding_mode='zeros')

        self.bn1 = nn.BatchNorm2d(8)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1, padding_mode='zeros')

        self.bn2 = nn.BatchNorm2d(16)

        self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1, padding_mode='zeros')

        self.bn3 = nn.BatchNorm2d(32)
        
        self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1, padding_mode='zeros')

        self.bn4 = nn.BatchNorm2d(64)

        self.conv5 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, padding=1, padding_mode='zeros')
        
        self.bn5 = nn.BatchNorm2d(32)

        self.fc1 = nn.Linear(in_features=32, out_features=16)
        self.fc2 = nn.Linear(in_features=16, out_features=10)
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.pool(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.pool(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.pool(x)

        x = self.conv4(x)
        x = self.bn4(x)
        x = F.relu(x)
        x = self.pool(x)

        x = self.conv5(x)
        x = self.bn5(x)
        x = F.relu(x)
        x = self.pool(x)
        
        x = x.reshape(-1, 32)

        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

In [9]:
net = MyLayer().to(device)

In [10]:
criterion = nn.CrossEntropyLoss()

# ИЗМЕНЕНИЕ
# optimizer = optim.SGD(net.parameters(), lr=1e-3, momentum=.9)
optimizer = optim.Adam(net.parameters())

In [11]:
%%time

num_epochs = 15

for epoch in range(num_epochs):
    running_loss = 0.0
    running_items = 0.0
    
    for i, data in enumerate(train_loader):
        
        inputs, labels = data[0].to(device), data[1].to(device)
        
        optimizer.zero_grad()
        
        outputs = net(inputs)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
        running_items += len(labels)
        
    print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
          f'Step [{i + 1}/{len(train_loader)}]. ' \
          f'Loss: {running_loss / running_items:.3f}')
    running_loss, running_items = 0.0, 0.0

print('Training is finished!')

Epoch [1/15]. Step [391/391]. Loss: 0.012
Epoch [2/15]. Step [391/391]. Loss: 0.008
Epoch [3/15]. Step [391/391]. Loss: 0.007
Epoch [4/15]. Step [391/391]. Loss: 0.007
Epoch [5/15]. Step [391/391]. Loss: 0.006
Epoch [6/15]. Step [391/391]. Loss: 0.006
Epoch [7/15]. Step [391/391]. Loss: 0.005
Epoch [8/15]. Step [391/391]. Loss: 0.005
Epoch [9/15]. Step [391/391]. Loss: 0.005
Epoch [10/15]. Step [391/391]. Loss: 0.005
Epoch [11/15]. Step [391/391]. Loss: 0.004
Epoch [12/15]. Step [391/391]. Loss: 0.004
Epoch [13/15]. Step [391/391]. Loss: 0.004
Epoch [14/15]. Step [391/391]. Loss: 0.004
Epoch [15/15]. Step [391/391]. Loss: 0.004
Training is finished!
CPU times: user 1min 35s, sys: 521 ms, total: 1min 35s
Wall time: 1min 35s


In [20]:
data_iter = iter(test_loader)
images, labels = data_iter.next()
images, labels = images.to(device), labels.to(device)

In [21]:
outputs = net(images)

In [22]:
_, preds = torch.max(outputs, 1)

In [23]:
true_names = np.array([classes[i] for i in labels])
pred_names = np.array([classes[i] for i in preds])

print(true_names)
print(pred_names)
print(f'Accuracy is {(true_names == pred_names).sum() / len(true_names)},\n {(true_names == pred_names).sum()} from {len(true_names)}')

['car' 'frog' 'car' 'horse' 'bird' 'car' 'car' 'truck']
['car' 'frog' 'car' 'horse' 'bird' 'truck' 'car' 'truck']
Accuracy is 0.875,
 7 from 8
